In [162]:
#read data(exclude replies)
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
import pandas as pd

text = pd.read_csv('Jan6(excludes replies).csv')
print('The number of tweets (excludes replies):',len(text))
#Text cleaning
import gensim
import gensim.corpora as corpora
import re
stemmer = SnowballStemmer('english')
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words += ['cultured','clean','cultivated','meat']
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['tweets']).split():
        if '@' not in words: #remove @users
            words = words.replace('#','') #remove hashtag symbol
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = words.lower()# lower form
                        words = re.sub(r'[^a-zA-Z]', ' ', words) #replace non-alphabets characters with space. From "can't" to "can t"
                        if len(words)>3:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if len(token) >1:
            if token not in stop_words:
                token = porter.stem(token) #root form
                doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]
text.to_csv('kk.csv')



import guidedlda
import numpy as np
import gensim
topic_tweets=[]
other_tweets=[]
topic_tweets_Company = []
other_tweets_processed = []
non_english_tweets = []

keywords_list = ['environ','environment','carbon','climat','greenhous','emiss','better','planet','save','reduc',
                   'system','futur','land','plastic','climatechang', #0 Environmental Impact
                   'plant','vegan','vegetarian','anim','altern','protein','plantbas', #1 Vegetarian
                   'animal','without','slaughter','cruelti','welfar','kill','cow','human','farm','save',#2 Animal Welfare
                   'consumpt','popul','demand','secur','wast','grow','global','sustain','need','solut',
                 'consum','eat','growth','feed','gluttoni',#3 Food Security
                   'antibiot','antimicrobi','resist','safeti','contamin','free','health','overus',
                 'diseas','use','metal','bacteria','regulatori','usda','fda','safe','joint','label',#4 Food Safety
                   'public','health','pandem','covid','risk','coronaviru',# 5 Public Health
                   'industri','suppli','convent','regular','system','chain','game','changer','compani',#6 Food Industry
                   'market','consum','store','groceri','product','demand','grow','launch','scale','cost',
                 'soon','sell', #7 Market
                   'seafood','sea','ocean','fish','aquacultur','shrimp','salmon','wild','lobster','tuna',
                 'sushi','crustacean',#8 Seafood
                   'tast','tender','textur','juici','test','delici','culinari','favorit',
                   'chicken','duck','poultri','egg','burger','steak','meatbal','beef','sausag','tasti',#9 Poultry and Meat
                   'stem','muscl','divis','biolog','cellular','technolog','agricultur','creat','made',
                 'lab','cell','biotech',#10 Process
                   'seri','fund','rais','invest','investor','dollar','pound','round','startup',
                   'opportun','first','seed','isra','acceler','announc','happi','thrill','excit','pleas','contribut',
                 'honor','proud','statement', #11 Fundraising Announcement
                   'thank','support','shoutout','help','shout','question','congrat','congratul',
                 'share','mention', #12 Appreciation
                   'confer','regist','live','symposium','stream','livestream','summit','co','founder',
                 'talk','speak','convers','ceo','tomorrow','tune', #13 Conference and Summit
                   'check','post','paper','interview','articl','blog','news','break','read',
                 'latest','watch','podcast','listen','episod','stori','campaign','coverag','video','singapor','discuss','vote','panel','chat',
                   'scientist','ceo','event','media','report','cover','bbc','present','innov', #14 Media
                   'hire','join','team','bring','welcom','work','appli','research','specialist',
                 'student','career','posit',#15 Hiring Information
                   'congrat','congratul','list','award','winner','finalist','partner','prize']#16 Congratulations
non_english_list = ['temiz','rkiy','erik','nda','konu','dan','da','ba','temiz','al','viand','para','na','dann','uft','laboratorio','dieser','kalbimi',
                   'restoranda','evento','mayo','komo','ind','tica','futuro','sonra','yla','cre','ili','daki',
                   'zaman']
for index,i in text.iterrows():
    count = 0
    non_english_count = 0
    for word in i['processed'].split():
        if word in keywords_list:
            count+=1 #if any keyword occur in a tweet, the tweet may belong to one of the above themes
        elif word in non_english_list:
            non_english_count+=1
    if count !=0: #if none of the keywords occur in a tweet, the tweet doesnot belong to any themes we found
        topic_tweets.append(i['processed'])
        topic_tweets_Company.append(i['Company'])
        
    elif non_english_count != 0:
        non_english_tweets.append(i['tweets'])
    else:
        other_tweets.append(i['tweets'])
        other_tweets_processed.append(i['processed'])
other = pd.DataFrame(data= [i for i in other_tweets],columns=['tweets'])
other['processed'] = [i for i in other_tweets_processed]

print('The number of tweets within 17 topics:',len(topic_tweets))
print('The number of other tweets:',len(other_tweets))
print('The number of non English tweets:',len(non_english_tweets))

#unigram LDA
from pprint import pprint
import gensim
import gensim.corpora as corpora
unigram = []
unigram_list = []
for index, i in other.iterrows():
    unigram=[]
    for word in i['processed'].split():
        unigram.append(word)
    unigram_list.append(unigram)
data_words = [i for i in unigram_list]
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, random_state=10,
                                            alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics(num_words=50,num_topics=2))


The number of tweets (excludes replies): 6501


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(1566 unique tokens: ['along', 'august', 'biggest', 'brought', 'celebr']...) from 760 documents (total 2689 corpus positions)
INFO:gensim.models.ldamodel:using autotuned alpha, starting with [0.5, 0.5]
INFO:gensim.models.ldamodel:using symmetric eta at 0.5
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 760 documents, updating model once every 760 documents, evaluating perplexity every 760 documents, iterating 50x with a convergence threshold of 0.001000


The number of tweets within 17 topics: 5667
The number of other tweets: 760
The number of non English tweets: 74


INFO:gensim.models.ldamodel:-8.250 per-word bound, 304.5 perplexity estimate based on a held-out corpus of 760 documents with 2689 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #760/760
INFO:gensim.models.ldamodel:optimized alpha [0.6629571, 0.6725566]
INFO:gensim.models.ldamodel:topic #0 (0.663): 0.009*"food" + 0.006*"great" + 0.005*"make" + 0.005*"year" + 0.005*"today" + 0.004*"look" + 0.004*"memphi" + 0.004*"meet" + 0.004*"time" + 0.004*"come"
INFO:gensim.models.ldamodel:topic #1 (0.673): 0.014*"cleanmeat" + 0.012*"food" + 0.008*"world" + 0.007*"gt" + 0.007*"futureoffood" + 0.005*"culturedmeat" + 0.005*"make" + 0.005*"great" + 0.004*"come" + 0.004*"time"
INFO:gensim.models.ldamodel:topic diff=0.660535, rho=1.000000
INFO:gensim.models.ldamodel:topic #0 (0.663): 0.009*"food" + 0.006*"great" + 0.005*"make" + 0.005*"year" + 0.005*"today" + 0.004*"look" + 0.004*"memphi" + 0.004*"meet" + 0.004*"time" + 0.004*"come" + 0.004*"think" + 0.003*"world" + 0.003*"chang" + 0.003*

[(0,
  '0.009*"food" + 0.006*"great" + 0.005*"make" + 0.005*"year" + 0.005*"today" '
  '+ 0.004*"look" + 0.004*"memphi" + 0.004*"meet" + 0.004*"time" + '
  '0.004*"come" + 0.004*"think" + 0.003*"world" + 0.003*"chang" + 0.003*"well" '
  '+ 0.003*"meat" + 0.003*"cleanmeat" + 0.003*"foodtech" + '
  '0.003*"futureoffood" + 0.003*"go" + 0.003*"good" + 0.003*"get" + '
  '0.003*"scienc" + 0.003*"would" + 0.003*"beyond" + 0.003*"best" + '
  '0.003*"peopl" + 0.002*"next" + 0.002*"know" + 0.002*"featur" + 0.002*"amaz" '
  '+ 0.002*"interest" + 0.002*"culturedmeat" + 0.002*"order" + 0.002*"arriv" + '
  '0.002*"sure" + 0.002*"pitch" + 0.002*"demo" + 0.002*"take" + 0.002*"still" '
  '+ 0.002*"wait" + 0.002*"inspir" + 0.002*"produc" + 0.002*"littl" + '
  '0.002*"love" + 0.002*"friend" + 0.002*"way" + 0.002*"start" + '
  '0.002*"answer" + 0.002*"result" + 0.002*"piec"'),
 (1,
  '0.014*"cleanmeat" + 0.012*"food" + 0.008*"world" + 0.007*"gt" + '
  '0.007*"futureoffood" + 0.005*"culturedmeat" + 0.005*"

In [163]:
seed_topic_list = [['environ','environment','carbon','climat','greenhous','emiss','better','planet','save','reduc',
                   'system','futur','land','ecolog','plastic','climatechang'], #0 Environmental Impact
                   ['plant','vegan','vegetarian','anim','altern','protein','plantbas'], #1 Vegetarian
                   ['animal','without','slaughter','cruelti','welfar','kill','cow','human','farm','save'],#2 Animal Welfare
                   ['consumpt','popul','demand','secur','wast','grow','global','sustain','need',
                   'consum','eat','growth','feed','gluttoni'],#3 Food Security
                   ['antibiot','antimicrobi','resist','safeti','contamin','free','health','overus',
                   'diseas','use','metal','bacteria','regulatori','usda','fda','safe','joint','label'],#4 Food Safety
                   ['regulatori','usda','fda','safe','joint','label'],#5 Regulations
                   ['public','health','pandem','covid','risk','coronaviru'],# 6 Public Health
                   ['industri','suppli','convent','regular','system','chain','game','changer','compani'],#7 Food Industry
                   ['market','consum','store','groceri','product','demand','grow','launch','scale','cost','soon','sell'], #8 Market
                   ['seafood','sea','ocean','fish','aquacultur','shrimp','salmon','wild','lobster','tuna','sushi','crustacean'],#9 Seafood
                   ['chicken','duck','poultri','egg','burger','steak','meatbal','beef','sausag',
                    'tast','tender','textur','juici','test','delici','culinari','favorit','tasti'],#10 Poultry and Meat
                   ['stem','muscl','divis','biolog','cellular','technolog','agricultur','creat','made','lab','cell','biotech','tech'],#11 Process
                   ['seri','fund','rais','invest','investor','dollar','pound','round','startup',
                   'opportun','first','seed','isra','acceler','announc','happi','thrill','excit','pleas','contribut','honor','proud','statement'], #12 Fundraising Announcement
                   ['thank','support','shoutout','help','shout','question','mention'], #13 Appreciation
                   ['confer','regist','live','symposium','stream','livestream','summit','co','founder','talk','speak','ceo','tomorrow'], #14 Conference and Summit
                   ['hire','join','team','bring','welcom','work','appli','research','specialist','student','career','posit'],#15 Hiring Information
                   ['congrat','congratul','list','award','winner','finalist','partner','prize'],#16 Congratulations
                   ['check','post','paper','interview','articl','blog','news','break','read',
                   'latest','watch','podcast','listen','episod','stori','coverag','campaign','video','singapor','discuss','vote','panel',
                    'chat','scientist','report','ceo','media','bbc','present','innov']]#17 Media

from sklearn.feature_extraction.text import CountVectorizer
model = guidedlda.GuidedLDA(n_topics=18,n_iter=200,random_state=1,refresh=10,alpha=0.1,eta=0.01)
vectorizer = CountVectorizer(min_df = 1)
X = vectorizer.fit_transform(topic_tweets)

vocab = vectorizer.get_feature_names()
word2id = dict((v,idx) for idx,v in enumerate(vocab))
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

model.fit(X.toarray(),seed_topics=seed_topics,seed_confidence=0.65)
topic_word = model.topic_word_
n_top_words = 40
vocab = tuple(vocab)

for i, topic_dist in enumerate(topic_word):
    print('\n')
    print('Topic:',i)
    words_probability = np.array(-topic_dist)
    for index in range(n_top_words):
        print(round(abs(np.sort(words_probability))[:(n_top_words)][index],4),'*',
              np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1][index],sep='',end='  ')

INFO:guidedlda:n_documents: 5667
INFO:guidedlda:vocab_size: 7255
INFO:guidedlda:n_words: 66323
INFO:guidedlda:n_topics: 18
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -782197
INFO:guidedlda:<10> log likelihood: -556313
INFO:guidedlda:<20> log likelihood: -543549
INFO:guidedlda:<30> log likelihood: -538755
INFO:guidedlda:<40> log likelihood: -536280
INFO:guidedlda:<50> log likelihood: -534708
INFO:guidedlda:<60> log likelihood: -533926
INFO:guidedlda:<70> log likelihood: -533306
INFO:guidedlda:<80> log likelihood: -532225
INFO:guidedlda:<90> log likelihood: -531855
INFO:guidedlda:<100> log likelihood: -531635
INFO:guidedlda:<110> log likelihood: -530764
INFO:guidedlda:<120> log likelihood: -530200
INFO:guidedlda:<130> log likelihood: -530314
INFO:guidedlda:<140> log likelihood: -530139
INFO:guidedlda:<150> log likelihood: -529612
INFO:guidedlda:<160> log likelihood: -529626
INFO:guidedlda:<170> log likelihood: -529222
INFO:guidedlda:<180> log likelihood: -528810
INFO:g



Topic: 0
0.0419*chang  0.0299*climat  0.0294*world  0.0279*anim  0.0272*food  0.0227*better  0.0155*could  0.0147*environ  0.0142*environment  0.014*futur  0.0127*agricultur  0.0125*produc  0.0122*impact  0.0122*eat  0.0117*water  0.0115*human  0.0115*less  0.011*planet  0.011*reduc  0.011*sustain  0.01*use  0.01*want  0.0097*love  0.009*earth  0.0087*climatechang  0.0087*livestock  0.0082*solv  0.0082*solut  0.0082*without  0.008*problem  0.008*convent  0.0077*land  0.0077*make  0.0075*system  0.0067*take  0.0067*issu  0.0067*emiss  0.0062*farm  0.006*diet  0.0057*real  

Topic: 1
0.0476*protein  0.0467*plant  0.044*food  0.0438*base  0.0386*altern  0.028*plantbas  0.0223*burger  0.0223*vegan  0.0174*industri  0.0152*beyond  0.012*next  0.0103*dairi  0.0101*cubiq  0.0101*foodtech  0.0098*anim  0.009*show  0.0084*offer  0.0079*go  0.0076*milk  0.0076*smart  0.0071*tast  0.0065*fat  0.0065*imposs  0.0063*get  0.0063*ingredi  0.0063*omega  0.0063*say  0.0063*invest  0.006*replac  0.005

0.0312*food  0.0293*speak  0.0287*futur  0.024*join  0.0212*panel  0.0191*founder  0.0181*confer  0.0175*discuss  0.0157*talk  0.0138*event  0.0126*tomorrow  0.0122*live  0.0122*week  0.0118*summit  0.0118*today  0.0112*innov  0.011*th  0.0104*ceo  0.01*come  0.0098*co  0.0094*present  0.0088*watch  0.0086*meet  0.0081*foodtech  0.0077*excit  0.0071*next  0.0069*goodfoodconfer  0.0069*pitch  0.0067*tune  0.0065*regist  0.0063*host  0.0061*onlin  0.0061*speaker  0.0057*june  0.0055*pm  0.0053*technolog  0.0053*leader  0.0053*cultivatedmeat  0.0051*hear  0.0049*happen  

Topic: 15
0.0355*team  0.0243*research  0.0201*work  0.0189*scienc  0.0184*food  0.0164*hire  0.0156*scientist  0.0154*excit  0.0149*join  0.0149*meat  0.0142*look  0.0124*engin  0.0124*develop  0.0114*welcom  0.0112*appli  0.0109*pleas  0.0102*futur  0.0099*co  0.0094*make  0.0079*meet  0.0077*check  0.0077*want  0.0075*learn  0.0075*member  0.0075*founder  0.0072*board  0.0072*role  0.007*cultur  0.0067*today  0.0067*p

In [160]:
other_retweet = []
other_not_retweet = []
other_retweet_processed = []
other_not_retweet_processed = []
for index,i in other.iterrows():
    if i['tweets'].startswith('RT:'):
        other_retweet.append(i['tweets'])
        other_retweet_processed.append(i['processed'])
    else:
        other_not_retweet.append(i['tweets'])
        other_not_retweet_processed.append(i['processed'])
print('the number of retweets in Other:',len(other_retweet))
number = 0
for i in other_not_retweet_processed:
    if len(i.split())<10:
        number +=1
print('the number of tweets that are less than 10 words:',number)
        
    

the number of retweets in Other: 281
the number of tweets that are less than 10 words 461
